In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import timeit
from sklearn.cluster import KMeans
from sklearn.neighbors import KDTree
from sklearn import datasets

In [16]:
## Coreset Algorithm
clusters = {}
centroids = []
np_data = []
total_for_coreset = 0
#function to generate 'num_points' random points of 'dim' dimensions.
def generate_data(data, m):
	dim = data.shape[1]
	rows = data.shape[0]
	data_mean = np.mean(data, axis = 0)
	distance = 0
	for point in data:
		distance += np.sum(np.square(point-data_mean))
	prob_dist = []
	#calculating proposal  distribution for each row
	for point in data:
		value=((0.5*(1/n))+0.5*((np.sum(np.square(point-data_mean)))/distance))
		prob_dist.append(value)
	df = pd.DataFrame(prob_dist)
	data = pd.DataFrame(data)
	data['Prob_dist'] = df
	weight_value = []
	for i in range(n):
		weight = 1/(m*prob_dist[i])
		weight_value.append(weight)
	df = pd.DataFrame(weight_value)
	data['weight_value'] = df
	sampled_points = data.sample(n=m, frac=None, replace=False, weights='weight_value', random_state=None, axis=None)
	return sampled_points.iloc[:,:dim]

In [17]:
data = pd.read_csv('kddcup.csv',sep=",")
data = np.array(data)
data = data[:,4:40]
# Identifying data length
n = data.shape[0]
k = 23
## Finding the number of attributes.
features = data.shape[1]
start = timeit.default_timer()
points = np.array(generate_data(data, k))
stop = timeit.default_timer()
total_for_coreset = stop-start

## Running KMeans on train data.
start = timeit.default_timer()
y_predict = KMeans(n_clusters = k, init = points, n_init =1).fit(data)
stop = timeit.default_timer()
time_for_kmeans = stop-start

In [18]:
# No. of. Nearest Neighbors
n_n = 7

## Storing Cluster centers in cluster_centers
cluster_centers = np.array(y_predict.cluster_centers_)

## generating Query point
query = []
for i in range(0,features):
    x = round(random.random()*random.choice([10,100,1000]),1)
    query.append(x)

query = np.array(query)
query = query.reshape(1,features)


## Finding the nearest cluster to query point.
    ## Finding the euclidean distance from first cluster_center point.
start = timeit.default_timer()
dist = 0
for i in range(features):
   dist += (query[0,i]-cluster_centers[0,i])**2
dist = np.sqrt(dist)
near_cluster = 0
i = 1

 ## Finding the euclidean distance from other cluster_center points.
while i < k:
    dist_j = 0
    for j in range(features):
       dist_j += (query[0,j]-cluster_centers[i,j])**2
    dist_j = np.sqrt(dist_j)
    if(dist_j < dist):
        dist = dist_j
        near_cluster = i
    i += 1
stop = timeit.default_timer()   
time_for_closest_center = stop-start   
## Finding the cluster elements to construct KD-Tree

cluster_values = np.ones([n,features])
j = 0
for i in range(n):
    if y_predict.labels_[i] == near_cluster:
        cluster_values[j] = data[i]
        j += 1
cluster_values = cluster_values[:j,:j]

## KDTree Construction and Querying
start = timeit.default_timer()
tree = KDTree(cluster_values)
stop = timeit.default_timer()
time_for_kdtree = stop-start

start = timeit.default_timer()
dist_main , ind_main = tree.query(query[0:1],n_n)
stop = timeit.default_timer()
query_time = stop-start


j=0
for i in ind_main:
    print(cluster_values[i], dist_main[j])
    j += 1
print(total_for_coreset,time_for_kmeans,time_for_closest_center,time_for_kdtree,query_time)
print(total_for_coreset + time_for_kmeans + time_for_closest_center + time_for_kdtree + query_time)

[[6.76e+02 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 1.00e+00 1.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  1.00e+00 0.00e+00 0.00e+00 1.00e+00 4.00e+00 1.00e+00 0.00e+00 1.00e+00
  5.00e-01 0.00e+00 0.00e+00 0.00e+00]
 [6.41e+02 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 1.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 2.00e+00 2.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  1.00e+00 0.00e+00 0.00e+00 9.00e+00 5.00e+00 5.60e-01 2.20e-01 5.60e-01
  0.00e+00 0.00e+00 0.00e+00 0.00e+00]
 [5.67e+02 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 1.00e+00
  0.00e+00 0.00e+00 0.00e+00 6.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 1.00e+00 2.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  1.00e+00 0.00e+00 1.00e+00 1.00e+00 2.10e+01 1.00e+00 0.00e+00 1.00e+00
  1.00e-01 0.00e+00 0.00e+00 0.00e

In [19]:
cluster_values.shape

(190107, 36)

In [12]:
start = timeit.default_timer()
tree = KDTree(data)
stop = timeit.default_timer()
total_time = stop-start
print(total_time)
j=0
start = timeit.default_timer()
dist_dum , ind_dum = tree.query(query,n_n)
stop = timeit.default_timer()
total_time = stop-start
print(total_time)
for i in ind_dum:
    print(data[i],dist_dum[j])
    j =+ 1

1115.123546068
0.09504488299899094
[[8 640 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0.0 0.0 0.0 0.0 1.0 0.0 0.0
  17 1 0.06 0.29 0.06 0.0 0.0 0.0 0.0]
 [7 674 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0.0 0.0 0.0 0.0 1.0 0.0 0.0
  77 4 0.03 0.04 0.01 0.5 0.0 0.0 0.0]
 [188 631 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0.0 0.0 0.0 0.0 1.0 0.0 0.0
  25 44 1.0 0.0 0.04 0.07 0.0 0.0 0.0]
 [8 731 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0.0 0.0 0.0 0.0 1.0 0.0 0.0 2
  2 0.5 1.0 0.5 1.0 0.0 0.0 0.0]
 [8 731 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0.0 0.0 0.0 0.0 1.0 0.0 0.0
  16 3 0.19 0.19 0.06 0.0 0.0 0.0 0.0]
 [205 654 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 5 5 0.0 0.0 0.0 0.0 1.0 0.0 0.0
  5 16 1.0 0.0 0.2 0.19 0.0 0.0 0.0]
 [8 731 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 1 0.0 0.0 0.0 0.0 0.5 1.0 0.0
  30 6 0.2 0.07 0.03 0.0 0.0 0.0 0.0]] [2166.5087088  2167.92949392 2167.98298114 2168.0339942  2168.13422689
 2168.31895903 2168.44830508]


In [13]:
# Loading data
data = pd.read_csv('kddcup.csv',sep=",")
data = np.array(data)
data = data[:,4:40]
# Identifying data length
n = data.shape[0]
k = 23

# No. of. Nearest Neighbors
n_n = 7

## Finding the number of attributes.
features = data.shape[1]


## Running KMeans on train data.
start = timeit.default_timer()
y_predict = KMeans(n_clusters = k).fit(data)
stop = timeit.default_timer()
time_for_kmeans = stop-start
## Storing Cluster centers in cluster_centers
cluster_centers = np.array(y_predict.cluster_centers_)


## Finding the nearest cluster to query point.
    ## Finding the euclidean distance from first cluster_center point.
start = timeit.default_timer()
dist = 0
for i in range(features):
   dist += (query[0,i]-cluster_centers[0,i])**2
dist = np.sqrt(dist)
near_cluster = 0
i = 1

 ## Finding the euclidean distance from other cluster_center points.
while i < k:
    dist_j = 0
    for j in range(features):
       dist_j += (query[0,j]-cluster_centers[i,j])**2
    dist_j = np.sqrt(dist_j)
    if(dist_j < dist):
        dist = dist_j
        near_cluster = i
    i += 1
    
stop = timeit.default_timer()   
time_for_closest_center = stop-start
## Finding the cluster elements to construct KD-Tree
cluster_values = np.ones([n,features])
j = 0
for i in range(n):
    if y_predict.labels_[i] == near_cluster:
        cluster_values[j] = data[i]
        j += 1
cluster_values = cluster_values[:j,:j]

## KDTree Construction and Querying
start = timeit.default_timer()
tree = KDTree(cluster_values)
stop = timeit.default_timer()
time_for_kdtree = stop-start

start = timeit.default_timer()
dist_main , ind_main = tree.query(query[0:1],n_n)
stop = timeit.default_timer()
query_time = stop-start


j=0
for i in ind_main:
    print(cluster_values[i], dist_main[j])
    j += 1
print(time_for_kmeans,time_for_closest_center,time_for_kdtree,query_time)
print(time_for_closest_center + time_for_kdtree + query_time)

[[8.00e+00 6.40e+02 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 1.00e+00 1.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  1.00e+00 0.00e+00 0.00e+00 1.70e+01 1.00e+00 6.00e-02 2.90e-01 6.00e-02
  0.00e+00 0.00e+00 0.00e+00 0.00e+00]
 [7.00e+00 6.74e+02 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 1.00e+00 1.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  1.00e+00 0.00e+00 0.00e+00 7.70e+01 4.00e+00 3.00e-02 4.00e-02 1.00e-02
  5.00e-01 0.00e+00 0.00e+00 0.00e+00]
 [1.88e+02 6.31e+02 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 1.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 1.00e+00 1.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  1.00e+00 0.00e+00 0.00e+00 2.50e+01 4.40e+01 1.00e+00 0.00e+00 4.00e-02
  7.00e-02 0.00e+00 0.00e+00 0.00e

In [14]:
cluster_values.shape

(465629, 36)